In [217]:
# GENERAL USE LIBRARY IMPORTS

import datetime as dt
import random
import random as rand
import warnings
from datetime import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import pandas as pd
import os
import requests
from zipfile import ZipFile

warnings.filterwarnings('ignore')

In [74]:
# Create dictionary of team abbvs and names
team_abbvs = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK',  'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC','SAS', 'TOR', 'UTA','WAS']
team_names = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets','Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']
name_to_abbv = dict(zip(team_names,team_abbvs))

In [3]:
import requests
# Read the Data
url = 'https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba-odds-2012-13/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url,headers=header)

season_game_results = pd.read_html(r.text)[0]

# Make first row Column Names
season_game_results.columns = season_game_results.iloc[0]
# Delete first row
season_game_results = season_game_results[1:]
# Drop columns
season_game_results.drop(columns=['Date', 'Rot','1st', '2nd', '3rd', '4th', 'Open', 'Close', '2H'], inplace=True)
# Rename Columns
column_name_changes = {
    'Team': 'Away_Team',
    'Final': 'A_Final_Score'}
season_game_results.rename(columns=column_name_changes, inplace=True)

season_game_results['Home_Team'] = season_game_results.apply(lambda row: row['Away_Team'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['H_Final_Score'] = season_game_results.apply(lambda row: row['A_Final_Score'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['ML'] = season_game_results['ML'].shift(-1)
season_game_results = season_game_results[~(season_game_results.index % 2 == 0)]
season_game_results['W/L'] = season_game_results.apply(lambda row: 'W' if row['H_Final_Score'] >= row['A_Final_Score'] else 'L', axis=1)

season_game_results = season_game_results.reset_index(drop=True)


# Display df
season_game_results

,VH,Away_Team,A_Final_Score,ML,Home_Team,H_Final_Score,W/L
0,V,Washington,84,-250,Cleveland,94,W
1,V,Boston,107,-280,Miami,120,W
2,V,Dallas,99,-500,LALakers,91,L
3,V,Indiana,90,100,Toronto,88,L
4,V,Denver,75,110,Philadelphia,84,W
...,...,...,...,...,...,...,...
1309,V,Miami,77,-125,SanAntonio,113,L
1310,V,Miami,109,110,SanAntonio,93,W
1311,V,Miami,104,-110,SanAntonio,114,W
1312,V,SanAntonio,100,-280,Miami,103,W


In [4]:
# PRESEASON TITLE ODDS
def get_title_odds(year):
    title_odds_ = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{year}_preseason_odds.html')
    title_odds_  = title_odds_[0]
    title_odds_['year'] = year
    t_columns_to_drop = ['Unnamed: 2','W-L O/U','Result']
    title_odds_ = title_odds_.drop(t_columns_to_drop,axis=1)
    title_odds_['Abbv'] = title_odds_['Team'].map(name_to_abbv)
    q=10 # Desired number of teams to analyze
    title_odds_ = title_odds_.sort_values(by='Odds',ascending=True).head(q)
    return title_odds_

In [5]:
title_odds_list = []
for year in range(2013,2024):
    title_odds_ = get_title_odds(year)
    title_odds_list.append(title_odds_)
title_odds_df = pd.concat(title_odds_list,ignore_index=True)   

In [6]:
title_odds_df[title_odds_df['year'] == 2013]

,Team,Odds,year,Abbv
0,Miami Heat,225,2013,MIA
1,Los Angeles Lakers,275,2013,LAL
2,Oklahoma City Thunder,450,2013,OKC
3,Chicago Bulls,1200,2013,CHI
4,San Antonio Spurs,1600,2013,SAS
5,Boston Celtics,2000,2013,BOS
6,Los Angeles Clippers,2200,2013,LAC
7,Brooklyn Nets,2800,2013,BRK
8,Indiana Pacers,3300,2013,IND
9,Memphis Grizzlies,3300,2013,MEM


In [7]:
# BOX PLUS/MINUS 
box_plus_minus = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2013_advanced.html')
box_plus_minus = box_plus_minus[0]
box_cols_to_drop = ['Rk', 'Age', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'VORP']
box_plus_minus = box_plus_minus.drop(box_cols_to_drop,axis=1)

#Delete Jr. from the names of the players
text_to_delete = ' Jr.'
box_plus_minus['Player'] = box_plus_minus['Player'].str.replace(text_to_delete,'')

# Function to rewrite the Names - Works for individual teams but not in the whole list?
def custom_short_name(full_name):
    first_name, last_name = full_name.split(maxsplit = 1)
    return first_name[0] + '. ' + last_name

box_plus_minus

,Player,Pos,Tm,BPM
0,Quincy Acy,PF,TOR,0.1
1,Jeff Adrien,PF,CHA,-3.0
2,Arron Afflalo,SF,ORL,-2.9
3,Josh Akognon,PG,DAL,-3.0
4,Cole Aldrich,C,TOT,-2.9
...,...,...,...,...
591,Nick Young,SG,PHI,-0.7
592,Sam Young,SG,IND,-2.7
593,Thaddeus Young,PF,PHI,1.2
594,Luke Zeller,C,PHO,-7.2


In [14]:
# MIA - Getting Starting Lineups Data
MIA13 = pd.read_html('https://www.basketball-reference.com/teams/MIA/2013_start.html')
MIA13 = MIA13[0]
MIA13 = MIA13.set_index('G')
columns_to_drop = ['Date','Unnamed: 2','Unnamed: 3','Unnamed: 4','Opponent','Unnamed: 8','Tm','Opp','W','L']
MIA13 = MIA13.drop(columns_to_drop,axis=1)
MIA13 = MIA13.fillna('Home')
new_column_names = {'Unnamed: 5': 'Home', 'Unnamed: 7': 'Results'}
MIA13 = MIA13.rename(columns=new_column_names)
MIA13  = MIA13[MIA13['Home'] == 'Home']
MIA13[['SG', 'PF','C','PG','SF']] = MIA13['Starting Lineup'].str.split('·', n=4, expand=True)
# Define the choices to count
MIA_starting_lineup = MIA13.iloc[0, 2].split('·')
MIA_starting_lineup = [MIA_starting_lineup.strip() for MIA_starting_lineup in MIA_starting_lineup]
MIA13 = MIA13.drop(columns='Starting Lineup',axis=1)

# Remove the leading spaces
MIA13 = MIA13.applymap(lambda x: x.lstrip() if isinstance(x, str) else x)

# Remove the occurrence of \xa0 (non-breaking space) 
MIA13 = MIA13.replace('\xa0', '', regex=True)

# Count occurrences of choices across columns and sum the counts in a new column
MIA13['SL Count'] = (MIA13[['SG', 'PF','C','PG','SF']].isin(MIA_starting_lineup)).sum(axis=1)/5


# Display the updated DataFrame and Results
print('Pct of starting lineup in Losses vs Wins', MIA13.groupby('Results')['SL Count'].mean())

# Create BPM dataframe for each team
MIA_bpm = box_plus_minus[box_plus_minus['Tm'] == 'MIA']
MIA_bpm['Player'] = MIA_bpm['Player'].apply(custom_short_name)
MIA_bpm['BPM'] = MIA_bpm['BPM'].astype(float)

# Starting Lineup BPM
MIA_start_bpm = MIA_bpm.loc[MIA_bpm['Player'].isin(MIA_starting_lineup),'BPM'].sum()

# Function to calculate sum of BPM values for players in the same row
def calculate_bpm_sum(row):
    return MIA_bpm[MIA_bpm['Player'].isin(row)].loc[:, 'BPM'].sum()

# Apply the function to the DataFrame to get the sum of BPM values for each row
MIA13['BPM_Sum'] = MIA13.apply(calculate_bpm_sum, axis=1)
MIA13['BPM_Pct'] = MIA13['BPM_Sum'].div(MIA_start_bpm).round(2)
MIA13.head()


Pct of starting lineup in Losses vs Wins Results
L    0.650000
W    0.794595
Name: SL Count, dtype: float64


,Home,Results,SG,PF,C,PG,SF,SL Count,BPM_Sum,BPM_Pct
G,,,,,,,,,,
1,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
3,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
4,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
5,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
12,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0


In [11]:
# Function for Betting $100
def bet_100(row):
    if row['Sign'] > 0:
        if row['Results'] == 'W':
            return 100*(-100)/row['ML']
        else:
            return -100
    else:
        if row['Results'] == 'W':
            return row['ML']
        else:
            return -100
# Function for Betting the MoneyLine        
def bet_ml(row):
    if row['Sign'] > 0:
        if row['Results'] == 'W':
            return 100
        else:
            return row['ML']
    else:
        if row['Results'] == 'W':
            return row['ML']
        else:
            return -100    

In [115]:
url_base = "https://www.basketball-reference.com/boxscores/"
date = "20240421"  ## yyyymmdd formatted
team_abbr = "BOS"
formatted_url = "{}{}0{}.html".format(url_base, date, team_abbr)
print(formatted_url)

https://www.basketball-reference.com/boxscores/202404210BOS.html


In [209]:

boxscore = pd.read_html('https://www.basketball-reference.com/boxscores/202404210BOS.html')
awayteam_basic = boxscore[0]
awayteam_advanced = boxscore[7]
hometeam_basic = boxscore[8]
hometeam_advanced = boxscore[15]

#print(awayteam_basic)
awayteam = pd.merge(awayteam_basic,awayteam_advanced,left_index= True,right_index =True)
awayteam = awayteam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
awayteam = awayteam.rename(columns={'Starters': 'Players'})
awayteam.columns = awayteam.columns.droplevel(0)
hometeam = pd.merge(hometeam_basic,hometeam_advanced,left_index= True,right_index =True)
hometeam = hometeam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
hometeam = hometeam.rename(columns={'Starters': 'Players'})
hometeam.columns = hometeam.columns.droplevel(0)
hometeam

,Players,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,Jayson Tatum,41:08,7,18,.389,1,8,.125,8,8,...,26.9,15.0,37.8,2.7,0.0,12.2,28.9,125,107,3.1
1,Jaylen Brown,34:14,6,12,.500,3,5,.600,2,4,...,21.6,14.4,13.8,0.0,3.2,6.8,20.9,130,111,2.7
2,Kristaps Porziņģis,34:07,7,13,.538,4,8,.500,0,0,...,10.8,7.2,4.8,1.6,6.4,13.3,21.3,117,106,2.1
3,Derrick White,33:28,6,10,.600,4,8,.500,4,4,...,11.0,5.5,18.9,0.0,3.3,20.3,21.4,138,113,6.3
4,Jrue Holiday,33:27,2,8,.250,2,4,.500,0,0,...,18.4,12.9,11.9,3.3,6.5,11.1,13.0,103,101,1.6
6,Al Horford,25:40,4,8,.500,2,5,.400,0,0,...,24.0,16.8,0.0,0.0,0.0,0.0,15.1,135,113,-0.6
7,Payton Pritchard,24:10,3,7,.429,2,5,.400,0,0,...,15.3,10.2,30.1,0.0,0.0,0.0,14.0,153,115,3.3
8,Sam Hauser,13:46,4,6,.667,4,6,.667,0,0,...,0.0,4.5,13.9,4.0,0.0,0.0,21.1,187,108,27.0
9,Oshae Brissett,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
10,Svi Mykhailiuk,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play


In [210]:
response = requests.get('https://www.basketball-reference.com/boxscores/202404210BOS.html')
boxscore = pd.read_html(response.text)

In [212]:
awayteam_basic = boxscore[0]
awayteam_advanced = boxscore[7]
hometeam_basic = boxscore[8]
hometeam_advanced = boxscore[15]

#print(awayteam_basic)
awayteam = pd.merge(awayteam_basic,awayteam_advanced,left_index= True,right_index =True)
awayteam = awayteam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
awayteam = awayteam.rename(columns={'Starters': 'Players'})
awayteam.columns = awayteam.columns.droplevel(0)
hometeam = pd.merge(hometeam_basic,hometeam_advanced,left_index= True,right_index =True)
hometeam = hometeam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
hometeam = hometeam.rename(columns={'Starters': 'Players'})
hometeam.columns = hometeam.columns.droplevel(0)
hometeam

,Players,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,Jayson Tatum,41:08,7,18,.389,1,8,.125,8,8,...,26.9,15.0,37.8,2.7,0.0,12.2,28.9,125,107,3.1
1,Jaylen Brown,34:14,6,12,.500,3,5,.600,2,4,...,21.6,14.4,13.8,0.0,3.2,6.8,20.9,130,111,2.7
2,Kristaps Porziņģis,34:07,7,13,.538,4,8,.500,0,0,...,10.8,7.2,4.8,1.6,6.4,13.3,21.3,117,106,2.1
3,Derrick White,33:28,6,10,.600,4,8,.500,4,4,...,11.0,5.5,18.9,0.0,3.3,20.3,21.4,138,113,6.3
4,Jrue Holiday,33:27,2,8,.250,2,4,.500,0,0,...,18.4,12.9,11.9,3.3,6.5,11.1,13.0,103,101,1.6
6,Al Horford,25:40,4,8,.500,2,5,.400,0,0,...,24.0,16.8,0.0,0.0,0.0,0.0,15.1,135,113,-0.6
7,Payton Pritchard,24:10,3,7,.429,2,5,.400,0,0,...,15.3,10.2,30.1,0.0,0.0,0.0,14.0,153,115,3.3
8,Sam Hauser,13:46,4,6,.667,4,6,.667,0,0,...,0.0,4.5,13.9,4.0,0.0,0.0,21.1,187,108,27.0
9,Oshae Brissett,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play
10,Svi Mykhailiuk,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play


In [129]:
hometeam.to_csv('hometest.csv', index=False)


In [178]:
hometeam.dtypes

Players    object
MP         object
FG         object
FGA        object
FG%        object
3P         object
3PA        object
3P%        object
FT         object
FTA        object
FT%        object
ORB        object
DRB        object
TRB        object
AST        object
STL        object
BLK        object
TOV        object
PF         object
PTS        object
+/-        object
MP         object
TS%        object
eFG%       object
3PAr       object
FTr        object
ORB%       object
DRB%       object
TRB%       object
AST%       object
STL%       object
BLK%       object
TOV%       object
USG%       object
ORtg       object
DRtg       object
BPM        object
dtype: object

In [192]:
df_target = pd.DataFrame(columns=hometeam.columns)

# extract last row (team data)
new_row = hometeam.iloc[-1]
new_row_df = new_row.to_frame().T

# Optionally, reset the index if you don't want the original index
new_row_df.reset_index(drop=True, inplace=True)

df_target = pd.concat([df_target, new_row_df ], ignore_index=True)
print(df_target)

       Players   MP  FG FGA   FG%  3P 3PA   3P%  FT FTA  ...  DRB%  TRB%  \
0  Team Totals  240  39  82  .476  22  49  .449  14  16  ...  87.2  56.4   

   AST% STL%  BLK%  TOV%   USG%   ORtg   DRtg  BPM  
0  69.2  6.9  13.6  10.1  100.0  131.7  108.6  NaN  

[1 rows x 37 columns]


In [132]:
# team stats
team_stats = pd.DataFrame(columns = ['Team', 'FG%', '3P%', 'FT%', 'TOVs forced',
                                     'TOV', 'PPG', 'TS%', 'eFG%', 'TRB%', 'TOV%',
                                     'ORtg', 'DRtg'])



In [213]:
def get_boxscore(response):
    
    boxscore = pd.read_html(response)
    # differing layouts if ot
    num = len(boxscore)
    num_div_two = int(num/2)
    awayteam_basic = boxscore[0]
    awayteam_advanced = boxscore[num_div_two -1]
    hometeam_basic = boxscore[num_div_two]
    hometeam_advanced = boxscore[num-1]
    
    awayteam = pd.merge(awayteam_basic,awayteam_advanced,left_index= True,right_index =True)
    awayteam = awayteam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
    awayteam = awayteam.rename(columns={'Starters': 'Players'})
    awayteam.columns = awayteam.columns.droplevel(0)
    
    hometeam = pd.merge(hometeam_basic,hometeam_advanced,left_index= True,right_index =True)
    hometeam = hometeam.drop(5,axis = 0).drop(columns = ('Unnamed: 0_level_0_y','Starters'),axis = 1)
    hometeam = hometeam.rename(columns={'Starters': 'Players'})
    hometeam.columns = hometeam.columns.droplevel(0)

    return awayteam, hometeam

In [67]:
# List of all months for NBA games in the season
months = ['october', 'november', 'december', 'january', 'february', 'march']

# Base URL for fetching data
base_url = "https://www.basketball-reference.com/leagues/NBA_2024_games-{}.html"

# Initialize an empty list to store DataFrames
full_schedule = []

# Loop through each month and fetch game data
for month in months:
    try:
        # Format URL for the current month
        url = base_url.format(month)
        
        # Read HTML table data
        month_schedule = pd.read_html(url)[0]  # Assumes the first table is the relevant one
        
        # Append the DataFrame to the full_schedule list
        full_schedule.append(month_schedule)
    except Exception as e:
        print(f"Failed to fetch data for {month}: {e}")

# Concatenate all DataFrames into one
full_schedule_df = pd.concat(full_schedule, ignore_index=True)

# Print or inspect the full schedule DataFrame
print(full_schedule_df)

                   Date Start (ET)        Visitor/Neutral  PTS  \
0     Tue, Oct 24, 2023      7:30p     Los Angeles Lakers  107   
1     Tue, Oct 24, 2023     10:00p           Phoenix Suns  108   
2     Wed, Oct 25, 2023      7:00p        Houston Rockets   86   
3     Wed, Oct 25, 2023      7:00p         Boston Celtics  108   
4     Wed, Oct 25, 2023      7:00p     Washington Wizards  120   
...                 ...        ...                    ...  ...   
1111  Sun, Mar 31, 2024      7:00p       Dallas Mavericks  125   
1112  Sun, Mar 31, 2024      7:00p          Chicago Bulls  109   
1113  Sun, Mar 31, 2024      7:00p  Oklahoma City Thunder  113   
1114  Sun, Mar 31, 2024      7:00p  Golden State Warriors  117   
1115  Sun, Mar 31, 2024      9:00p              Utah Jazz  106   

                Home/Neutral  PTS.1 Unnamed: 6 Unnamed: 7  Attend.  \
0             Denver Nuggets    119  Box Score        NaN  19842.0   
1      Golden State Warriors    104  Box Score        NaN  18064.0 

In [69]:
full_schedule_df.dtypes
full_schedule_df['f_Date'] = pd.to_datetime(full_schedule_df['Date']).dt.strftime('%Y%m%d')

In [78]:
full_schedule_df['Home_abbr'] = full_schedule_df['Home/Neutral'].map(name_to_abbv)
full_schedule_df['Away_abbr'] = full_schedule_df['Visitor/Neutral'].map(name_to_abbv)


In [79]:
print(full_schedule_df)

                   Date Start (ET)        Visitor/Neutral  PTS  \
0     Tue, Oct 24, 2023      7:30p     Los Angeles Lakers  107   
1     Tue, Oct 24, 2023     10:00p           Phoenix Suns  108   
2     Wed, Oct 25, 2023      7:00p        Houston Rockets   86   
3     Wed, Oct 25, 2023      7:00p         Boston Celtics  108   
4     Wed, Oct 25, 2023      7:00p     Washington Wizards  120   
...                 ...        ...                    ...  ...   
1111  Sun, Mar 31, 2024      7:00p       Dallas Mavericks  125   
1112  Sun, Mar 31, 2024      7:00p          Chicago Bulls  109   
1113  Sun, Mar 31, 2024      7:00p  Oklahoma City Thunder  113   
1114  Sun, Mar 31, 2024      7:00p  Golden State Warriors  117   
1115  Sun, Mar 31, 2024      9:00p              Utah Jazz  106   

                Home/Neutral  PTS.1 Unnamed: 6 Unnamed: 7  Attend.  \
0             Denver Nuggets    119  Box Score        NaN  19842.0   
1      Golden State Warriors    104  Box Score        NaN  18064.0 

In [80]:
simple_games_df = full_schedule_df[['f_Date', 'Home_abbr', 'Away_abbr']].rename(columns={
    'f_Date': 'Date',
    'Home_abbr': 'Home',
    'Away_abbr': 'Away'
})

In [81]:
print(simple_games_df)

          Date Home Away
0     20231024  DEN  LAL
1     20231024  GSW  PHO
2     20231025  ORL  HOU
3     20231025  NYK  BOS
4     20231025  IND  WAS
...        ...  ...  ...
1111  20240331  HOU  DAL
1112  20240331  MIN  CHI
1113  20240331  NYK  OKC
1114  20240331  SAS  GSW
1115  20240331  SAC  UTA

[1116 rows x 3 columns]


In [223]:

# create list of dates:
# Define the start and end dates
start_date = '2023-12-16'
#testing with smaller set
#end_date = '2023-10-31'
end_date = '2024-03-31'

# Generate the range of dates
date_range = pd.date_range(start=start_date, end=end_date)

# Convert to list
date_list = list(date_range)

# format dates:
formatted_date_list = [date.strftime('%Y%m%d') for date in date_range]

# Print the formatted list of dates
print(formatted_date_list)
# Print the list of dates


['20231216', '20231217', '20231218', '20231219', '20231220', '20231221', '20231222', '20231223', '20231224', '20231225', '20231226', '20231227', '20231228', '20231229', '20231230', '20231231', '20240101', '20240102', '20240103', '20240104', '20240105', '20240106', '20240107', '20240108', '20240109', '20240110', '20240111', '20240112', '20240113', '20240114', '20240115', '20240116', '20240117', '20240118', '20240119', '20240120', '20240121', '20240122', '20240123', '20240124', '20240125', '20240126', '20240127', '20240128', '20240129', '20240130', '20240131', '20240201', '20240202', '20240203', '20240204', '20240205', '20240206', '20240207', '20240208', '20240209', '20240210', '20240211', '20240212', '20240213', '20240214', '20240215', '20240216', '20240217', '20240218', '20240219', '20240220', '20240221', '20240222', '20240223', '20240224', '20240225', '20240226', '20240227', '20240228', '20240229', '20240301', '20240302', '20240303', '20240304', '20240305', '20240306', '20240307', '20

In [215]:
def fetch_with_backoff(url, retries=5, backoff_in_seconds=1):
    for attempt in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:
            sleep_time = backoff_in_seconds * (2 ** attempt)  # Exponential backoff
            print(f"Rate limit exceeded. Retrying in {sleep_time} seconds...")
            time.sleep(sleep_time)
        else:
            response.raise_for_status()  # Raise an error for other status codes
    raise Exception("Failed to fetch data after retries.")

In [218]:
def fetch_with_retry_after(url):
    response = requests.get(url)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 60))  # Default to 60 seconds if header is missing
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
        time.sleep(retry_after)
        return fetch_with_retry_after(url)  # Recursively retry fetching
    response.raise_for_status()  # Raise an error for other status codes
    return response

In [224]:
def save_box_scores(formatted_date_list, simple_games_df, url_base):
    # Create a main directory to hold all data before zipping
    main_folder = 'NBA_Box_Scores'
    os.makedirs(main_folder, exist_ok=True)
    
    for date in formatted_date_list:
        games_df = simple_games_df[simple_games_df['Date'] == date]
        for index, row in games_df.iterrows():
            home_abbr = row['Home']
            away_abbr = row['Away']
            game_folder = f"{date}/{away_abbr}@{home_abbr}"  # Folder name format: YYYYMMDD/Away@Home
            full_folder_path = os.path.join(main_folder, game_folder)
            os.makedirs(full_folder_path, exist_ok=True)

            # Format the URL
            formatted_url = f"{url_base}{date}0{home_abbr}.html"
            # Fetch and save box scores
            try: 
                response = fetch_with_retry_after(formatted_url)
                
            # Save each team's box score in the specific game folder
                away_df,home_df = get_boxscore(response.text)
                away_df.to_csv(f"{full_folder_path}/away_team.csv", index=False)
                home_df.to_csv(f"{full_folder_path}/home_team.csv", index=False)
            except Exception as e:
                print(f"Error fetching data for URL {formatted_url}: {str(e)}")
            
    # Zip the entire directory
    with ZipFile(f"{main_folder}.zip", 'w') as zipf:
        for root, dirs, files in os.walk(main_folder):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(main_folder, '..')))

# Example usage
url_base = "https://www.basketball-reference.com/boxscores/"

save_box_scores(formatted_date_list, simple_games_df, url_base)

Rate limit exceeded. Retrying after 3600 seconds...


KeyboardInterrupt: 

In [200]:
# parse through dates to get box scores


url_base = "https://www.basketball-reference.com/boxscores/"
home_abbr = ""
formatted_url = f"{url_base}{date}0{home_abbr}.html'"
for date in formatted_date_list:
    games_df = simple_games_df[simple_games_df['Date'] == date]
    # looping through games now
    for index, row in games_df.iterrows():
        # get url
        home_abbr = row['Home']
        formatted_url = "{}{}0{}.html".format(url_base, date, home_abbr)
        #from url get box score
        response = fetch_with_backoff(formatted_url)
        
        
        
        

DEN
GSW
ORL
NYK
IND
CHO
MIA
TOR
BRK
MEM
CHI
UTA
SAS
LAC
MIL
LAL


In [201]:
all_players_stats = pd.DataFrame()

for date in formatted_date_list:
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        formatted_url = "{}{}0{}.html".format(url_base, date, home_abbr)

        # Fetch the box score data
        awayteam, hometeam = get_boxscore(formatted_url)  # Assuming this function returns two DataFrames

        # Process both teams
        for team_df in [awayteam, hometeam]:
            team_df['Date'] = date  # Add date to track games
            team_df['Games Played'] = 1  # Initialize games played

            # Update player stats in the master DataFrame
            for i, player_row in team_df.iterrows():
                player_name = player_row['Players']
                
                if player_name in all_players_stats.index:
                    # Player exists, update their stats
                    player_stats = all_players_stats.loc[player_name]
                    new_games_played = player_stats['Games Played'] + 1
                    
                    # Calculate new averages for each stat
                    for col in player_stats.index:
                        if col not in ['Players', 'Date', 'Games Played']:
                            all_players_stats.at[player_name, col] = (player_stats[col] * player_stats['Games Played'] + player_row[col]) / new_games_played
                    
                    # Update games played
                    all_players_stats.at[player_name, 'Games Played'] = new_games_played
                else:
                    # New player, add to DataFrame
                    player_row = player_row.to_frame().T  # Transpose to convert Series to DataFrame
                    all_players_stats = pd.concat([all_players_stats, player_row], axis=0)

# Output the updated stats DataFrame
print(all_players_stats)

HTTPError: HTTP Error 429: Too Many Requests